In [ ]:
!pip install opencv-python

In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
default_image_size = tuple((256, 256))


def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [3]:
image_list_a, label_list = [], []

totalImage = 0

root_dir = listdir("PlantVillage/")

for image_folder in root_dir :
    image_list = listdir(f"PlantVillage/{image_folder}")
    
    for image in image_list:
        image_name = f"PlantVillage/{image_folder}/"+image
        if isinstance(image_name, str) == True:
            if image_name.endswith(".jpg") == True or image_name.endswith(".JPG") == True:
                image_list_a.append(convert_image_to_array(image_name))
                label_list.append(image_folder)
        totalImage = totalImage+1
        if(totalImage>=10):
            
            break
print(len(label_list),len(np.unique(np.asarray(label_list))))
from sys import getsizeof
getsizeof(image_list_a)

24 15


264

In [4]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [5]:
image_list_a = np.asarray(image_list_a)
x_train, x_test, y_train, y_test = train_test_split(image_list_a, image_labels, test_size=0.2, random_state = 42)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [6]:
EPOCHS = 5
INIT_LR = 1e-3
BS = 32
width=256
height=256
depth=3
inputShape = (height, width, depth)
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]) 
chanDim = 1
model = Sequential()

model.add(Conv2D(4, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))

model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(524))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 4)       112       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 4)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)  

In [7]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [8]:
'''history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )
'''
history = model.fit(x_train, y_train, batch_size=2,
    validation_data=(x_test, y_test),
    epochs=EPOCHS, verbose=1
    )


Instructions for updating:
Use tf.cast instead.
Train on 19 samples, validate on 5 samples
Epoch 1/5
19/19 [==============================] - 5s 280ms/step - loss: 0.3386 - acc: 0.9158 - val_loss: 1.4025 - val_acc: 0.8667
Epoch 2/5
19/19 [==============================] - 2s 119ms/step - loss: 0.2251 - acc: 0.9228 - val_loss: 0.5949 - val_acc: 0.8933
Epoch 3/5
19/19 [==============================] - 2s 121ms/step - loss: 0.2479 - acc: 0.9368 - val_loss: 0.6475 - val_acc: 0.8933
Epoch 4/5
19/19 [==============================] - 2s 121ms/step - loss: 0.2031 - acc: 0.9439 - val_loss: 0.5597 - val_acc: 0.8667
Epoch 5/5
19/19 [==============================] - 2s 117ms/step - loss: 0.1865 - acc: 0.9404 - val_loss: 0.4752 - val_acc: 0.8800


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test, y_test)